Testing Notebook

This notebook will be used to import and run the programmed scripts to keep outputs contained into just this file.

0. Initial import of modules

In [1]:
import os
import torch
print(os.getcwd())
torch.manual_seed(42)

c:\Users\benjo\Documents\Projects\crypto-lstm-trader


In [2]:
from preprocessing.fetch_binance import fetch_binance
from preprocessing.prep_data import target_def, z_score_norm, window_creation, train_val_test_split
from datasets.sequence_dataset import SequenceDataset

1. Obtaining the data

In [3]:
df = fetch_binance(
    exchange_ticker='BTC/USDT',
    start_date='2019-01-01T00:00:00Z',
    timeframe='1h',
    cache_path=('./data/usd_btc_binance.csv'),
    max_age_hrs=3
)
df.head()

,open,high,low,close,volume
timestamp,,,,,
2019-01-01 00:00:00,3701.23,3713.00,3689.88,3700.31,686.367420
2019-01-01 01:00:00,3700.20,3702.73,3684.22,3689.69,613.539115
2019-01-01 02:00:00,3689.67,3695.95,3675.04,3690.00,895.302181
2019-01-01 03:00:00,3690.00,3699.77,3685.78,3693.13,796.714818
2019-01-01 04:00:00,3692.32,3720.00,3685.94,3692.71,1317.452909


2. Normalisation and Window Creation

In [4]:
df = target_def(df)
norm_df = z_score_norm(df, split_time='2023-01-01 00:00:00')

x, y = window_creation(norm_df, window_size=48)
x.shape, y.shape


(torch.Size([57769, 48, 5]), torch.Size([57769]))

3. Splitting data into test, validation and training sets

In [5]:
x_train, y_train, x_val, y_val,x_test, y_test = train_val_test_split(x, y)
x_lengths = map(len,[x_train,x_val,x_test])
y_lengths = map(len,[y_train,y_val,y_test])
print(list(x_lengths))
print(list(y_lengths))

[40438, 8665, 8666]
[40438, 8665, 8666]


In [6]:
print(x.shape())

TypeError: 'torch.Size' object is not callable

4. Wrap the data in custom datasets so torch dataloaders can be used

In [ ]:
from torch.utils.data import DataLoader
from datasets.sequence_dataset import SequenceDataset

batch_size= 64
train_loader= DataLoader(SequenceDataset(x_train,y_train),batch_size=batch_size, shuffle=True)
val_loader = DataLoader(SequenceDataset(x_val,y_val), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(SequenceDataset(x_test,y_test),batch_size=batch_size, shuffle=False)


5. Model set up and Training

In [ ]:
from models.lstm import LSTMClassifier

model = LSTMClassifier(input_size=5,hidden_size=64)